The following code is used to scrape data from the web using selenium.

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import requests
import json
import os
import re
import sys
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

# Set up the driver

DRIVER_PATH = './chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

# Open the website
driver.get('https://tanzenmed.com/dashboard/login')
# add delay to allow the page to load
time.sleep(3)

### Login

In [2]:
# load credentials from json file
with open('./tanzen_keys.json') as f:
    data = json.load(f)
    email = data['email']
    password = data['password']

# Login to the website
username = driver.find_element(By.XPATH, "//*[@id='email']")
username.send_keys(email)
passwrd = driver.find_element(By.ID, "password")
passwrd.send_keys(password)

# Click on the Sign-in button
driver.find_element(By.XPATH, '//*[@id="app"]/div/div/div/div/div/span/form/button').click()
time.sleep(1)

### Search criteria

In [4]:
# CLick on the Burger menu
driver.find_element(By.XPATH, '//*[@id="app"]/div/nav/div/ul[1]/li/a').click()
time.sleep(1)

# Click on the "View Data" button
driver.find_element(By.XPATH, '//*[@id="app"]/div/div[1]/ul/ul/li[8]/a').click()
time.sleep(1)

# # click close button
# driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div[1]/ul/li[2]/a/svg[1]').click()
# time.sleep(1)

# Enter Search query
search_query = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[1]/div[2]/div[1]/fieldset/div/div/input')
search_query.send_keys('UPMC')
time.sleep(1)

# Expand the show more dropdown
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/div/div[3]/div[1]/select')
# select the "50" option
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/div/div[3]/div[1]/select/option[4]').click()
time.sleep(1)

# Expand the session table show more dropdown
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div[3]/div[1]/select')
# select the "50" option
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div[3]/div[1]/select/option[3]').click()
time.sleep(1)

### Selecting the search criteria

In [11]:
import shutil

# set the download path
down_path = "C://Users/lucki/Downloads"

# make a directory for the data if it doesn't exist
if not os.path.exists(f'{down_path}/tanzen_data'):
    os.mkdir(f'{down_path}/tanzen_data')


# loop through the results and click on each checkbox
for i in range(16, 53):
    driver.find_element(By.XPATH, f'//*[@id="vgt-table"]/tbody[2]/tr[{i}]/td[1]').click()
    time.sleep(2)

    # get the username
    username = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/table/tbody[2]/tr[{i}]/td[2]/span').text
    time.sleep(2)
    print(f'{i} username: {username}')
    
    # get the lenght of session table
    session_table = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]').text
    session_table = session_table.split('entries')
    session_table = session_table[0].split('of')
    session_table = session_table[1].strip()
    session_table = int(session_table)

    # make a directory for each user
    os.mkdir(f'{down_path}/tanzen_data/{username}')


    # download the data for each session
    for j in range(1, session_table+1):
        print(f'sess id: {j}')

        # click on the session ID
        driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div[2]/table/tbody/tr[{j}]/td[9]/span/span').click()

        # wait for the file to load
        time.sleep(120)

        # click action button
        driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div[2]/table/tbody/tr[{j}]/td[11]/span/span/div/button').click()
        time.sleep(3)
    

        # click download button
        driver.find_element(By.XPATH, f'/html/body/div[2]/div/div[3]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div[2]/table/tbody/tr[{j}]/td[11]/span/span/div/ul/li[3]/a').click()
        time.sleep(5)

    # move the download csv files to the user directory
    for file in os.listdir(down_path):
        if file.endswith('.csv'):
            shutil.move(f'{down_path}/{file}', f'{down_path}/tanzen_data/{username}')


16 username: UPMC-S1-AD04
sess id: 1
sess id: 2
sess id: 3
sess id: 4
sess id: 5
sess id: 6
sess id: 7
sess id: 8
sess id: 9
sess id: 10
sess id: 11
17 username: UPMC-S1-AD05
sess id: 1
sess id: 2
sess id: 3
sess id: 4
sess id: 5
sess id: 6
sess id: 7
sess id: 8
sess id: 9
sess id: 10
sess id: 11
sess id: 12
sess id: 13
sess id: 14
sess id: 15
sess id: 16
sess id: 17
sess id: 18
sess id: 19
sess id: 20
sess id: 21
sess id: 22
sess id: 23
sess id: 24
sess id: 25
sess id: 26
